# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [60]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [2]:
# downloading nltk libraries

# # The line is commented to avoid reinstallation
# nltk.download("popular")

# # Done downloading collection popular

In [3]:
# load data from database
engine = create_engine('sqlite:///NesrCategories.db')
sql = 'SELECT * FROM CategorizedMessages'
df = pd.read_sql(sql, engine)
X = df.message
Y = df.iloc[:, 4:]

# Get Y labels
Y_labels = list(Y)

In [4]:
# Check
display(df.shape, X.shape, Y.shape, 
        df.head(1), X.head(2), Y.head(2))

(26216, 40)

(26216,)

(26216, 36)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [47]:
# Define some shortcuts
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tokenize(text):
    # Normalize text
    text = text.lower()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # Tokenize text
    tokens = word_tokenize(text)
    # lemmatize and Remove stop words
    tokens = [
        lemmatizer.lemmatize(word) for word in tokens if word not in stop_words
    ]
    # Return
    return tokens


def display_single_results(y_test, y_pred, labels=None):
    if labels is None:
        labels = np.unique(y_pred)
    print("Labels: ", labels)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    print("\nConfusion Matrix:\n", confusion_mat)
    accuracy = (y_pred == y_test).mean()
    print(f"\nAccuracy: {accuracy *100:5.2f}%\n")
    weig = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    micr = precision_recall_fscore_support(y_test, y_pred, average='micro')
    macr = precision_recall_fscore_support(y_test, y_pred, average='macro')
    #bina = precision_recall_fscore_support(y_test, y_pred, average='binary')
    print(f"{'Metric':<12}{'Weighted':>10}{'Micro':>10}{'Macro':>10}"
          )  #{'Binary':>10}")
    print(
        f"{'Precision:':<12}{weig[0] *100:9.2f}%{micr[0] *100:9.2f}%{macr[0] *100:9.2f}%"
    )
    print(
        f"{'Recall:':<12}{weig[1] *100:9.2f}%{micr[1] *100:9.2f}%{macr[1] *100:9.2f}%"
    )
    print(
        f"{'F Score:':<12}{weig[2] *100:9.2f}%{micr[2] *100:9.2f}%{macr[2] *100:9.2f}%"
    )

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### 3-1 Trying to build normal Sequence of ML
> *3-1-1 Trying on **one output** only*

In [6]:
# Extracting one output only
y = Y.iloc[:, 0]

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(n_estimators=100)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# display results
print('Scoring...')
display_single_results(y_test, y_pred)

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
Scoring...
Labels:  [0 1 2]

Confusion Matrix:
 [[ 637  857    1]
 [ 269 4743    5]
 [   1   35    6]]

Accuracy: 82.18%

Metric        Weighted     Micro     Macro
Precision:      80.77%    82.18%    68.13%
Recall:         82.18%    82.18%    50.48%
F Score:        80.41%    82.18%    54.77%


#### 3-1 Trying to build normal Sequence of ML
> *3-1-2 Trying on **multi-outputs**!*

In [7]:
# First, getting a sample of 500 items from the dataframe
# to make testing easier

def get_sample(data_frame, sample_percent=10):
    '''
    Takes a data_frame, takes amount of sample_percent% of it randomly,
    Then returns X and Y (for this project only)
    '''
    n = int(data_frame.shape[0] * sample_percent / 100)
    df = data_frame.sample(n)
    X = df.message
    Y = df.iloc[:, 4:]
    return X, Y

#Test
_X, _Y = get_sample(df, 5)
display(df.shape, _X.shape, _Y.shape, _X.head(2), _Y.head(2))

(26216, 40)

(1310,)

(1310, 36)

8982    WHAT WE CAN TAKE ACTION CONCERNING THE EARTHQU...
709     What is the address of the radio station? I as...
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
8982,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
709,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

# The normal classifier
clf_single = RandomForestClassifier(n_estimators=100)
# The multi_output classifier
clf = MultiOutputClassifier(clf_single)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

print('*Done*')
# # display results
# print('Scoring...')
# display_results(y_test, y_pred)

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
*Done*


In [13]:
y_pred.shape, y_test.shape

((197, 36), (197, 36))

In [46]:
# Scoring outputs

# y_pred[:,0]==y_test[:,0]
# (y_pred[:,0] == y_test[:,0]).mean()
# y_test.iloc[:,0]
# y_test.values[:,0]
accuracy = [[(y_pred[:, i] == y_test.values[:, i]).mean(),
             *precision_recall_fscore_support(
                 y_test.values[:, i], y_pred[:, i], average='weighted')]
            for i in range(y_pred.shape[1])]
accuracy = np.array(accuracy)[:, :-1]
accuracy = (accuracy * 10000).astype(int) / 100
pd.DataFrame(
    data=accuracy,
    index=list(y_test),
    columns=['Accuracy', 'Precision', 'Recall', 'F-score'])

,Accuracy,Precision,Recall,F-score
related,77.15,73.29,77.15,71.55
request,87.81,87.78,87.81,84.66
offer,100.00,100.00,100.00,100.00
aid_related,71.57,71.54,71.57,69.90
medical_help,91.87,84.41,91.87,87.98
medical_products,95.93,92.04,95.93,93.95
search_and_rescue,96.44,93.01,96.44,94.70
security,98.98,97.97,98.98,98.47
military,96.44,93.01,96.44,94.70
child_alone,100.00,100.00,100.00,100.00


In [57]:
# Defining a scoring function
def display_multi_output_results(y_test, y_pred, labels=None):
    # Scoring the outputs
    accuracy = [[(y_pred[:, i] == y_test.values[:, i]).mean(),
                 *precision_recall_fscore_support(
                     y_test.values[:, i],
                     y_pred[:, i],
                     average='weighted',
                     labels=np.unique(y_pred[:, i]))]
                for i in range(y_pred.shape[1])]
    accuracy = np.array(accuracy)[:, :-1]
    accuracy = (accuracy * 10000).astype(int) / 100

    print('Showing scores...')
    print('\nAverage scores for all indicators...')
    scores = pd.DataFrame(
        data=accuracy,
        index=list(y_test),
        columns=['Accuracy %', 'Precision %', 'Recall %', 'F-score %'])
    print(scores.mean(axis=0))
    print('\Detailed scores for each indicator...')
    display(scores)
    return scores

In [58]:
# Putting it all together

# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining vectorizers and classifiers
print('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

# The normal classifier
clf_single = RandomForestClassifier(n_estimators=100)
# The multi_output classifier
clf = MultiOutputClassifier(clf_single)

# train classifier
print('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# Scoring the outputs
display_multi_output_results(y_test, y_pred)

print('*Done*')

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     93.058611
Precision %    92.960000
Recall %       96.295833
F-score %      93.836944
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,77.15,73.17,78.35,71.76
request,90.35,91.36,90.35,88.75
offer,100.00,100.00,100.00,100.00
aid_related,73.60,75.21,73.60,72.78
medical_help,90.35,90.35,100.00,94.93
medical_products,95.43,95.43,100.00,97.66
search_and_rescue,95.93,95.93,100.00,97.92
security,97.96,97.96,100.00,98.97
military,96.95,96.95,100.00,98.45
child_alone,100.00,100.00,100.00,100.00


*Done*


In [55]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_estimators=100)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [59]:
# Trying on the pipeline

# taking 3% of the original dataframe for faster tests
_X, _Y = get_sample(df, 3)

# Split data to train and test
print('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(_X, _Y)

# Defining The full pipline
print('Defining the pipeline...')
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',
                      MultiOutputClassifier(
                          RandomForestClassifier(n_estimators=100)))])

# training the pipeline
print('Training the pipeline...')
pipeline.fit(X_train, y_train)

# predicting on test data
print('Performing pipeline predictions...')
y_pred = pipeline.predict(X_test)

# Scoring the outputs
display_multi_output_results(y_test, y_pred)

print('*Done*')

Splitting...
Defining the pipeline...
Training the pipeline...
Performing pipeline predictions...
Showing scores...

Average scores for all indicators...
Accuracy %     93.255000
Precision %    93.246944
Recall %       96.058611
F-score %      93.633611
dtype: float64
\Detailed scores for each indicator...


,Accuracy %,Precision %,Recall %,F-score %
related,77.15,74.60,77.94,73.16
request,83.75,86.42,83.75,77.26
offer,98.47,98.47,100.00,99.23
aid_related,73.60,74.13,73.60,71.48
medical_help,92.38,92.38,100.00,96.04
medical_products,93.40,93.40,100.00,96.58
search_and_rescue,97.46,97.46,100.00,98.71
security,99.49,99.49,100.00,99.74
military,97.96,97.96,100.00,98.97
child_alone,100.00,100.00,100.00,100.00


*Done*


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# I had a better output than the classification_report

### 6. Improve your model
Use grid search to find better parameters. 

In [63]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001EBC699B8C8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False,

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.